### Запуск QtLab

In [1]:
from qsweepy import *
from qsweepy.instruments import *
from qsweepy.save_pkl import *
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import interactive
import pandas as pd
interactive(True)
%matplotlib qt5


Failed loading module Signal_Hound_SA:  name 'signal_hound_dll' is not defined


In [2]:
#import resonator_quality_factor_fit
import imp
imp.reload(resonator_quality_factor_fit)
import qsweepy.save_pkl
imp.reload(qsweepy.save_pkl)
from qsweepy.save_pkl import *

In [3]:
rf_switch = nn_rf_switch('rf_switch', address='10.20.61.224')

D:\qtlab_replacement\qsweepy\instrument.py 290 ch1_switch
D:\qtlab_replacement\qsweepy\instrument.py 290 ch2_switch


In [4]:
pna = Agilent_N5242A('pna', address = 'PNA')

D:\qtlab_replacement\qsweepy\instrument.py 290 sweep_mode
D:\qtlab_replacement\qsweepy\instrument.py 290 nop
D:\qtlab_replacement\qsweepy\instrument.py 290 bandwidth
D:\qtlab_replacement\qsweepy\instrument.py 290 average_mode
D:\qtlab_replacement\qsweepy\instrument.py 290 averages
D:\qtlab_replacement\qsweepy\instrument.py 290 average
D:\qtlab_replacement\qsweepy\instrument.py 290 frequency
D:\qtlab_replacement\qsweepy\instrument.py 290 centerfreq
D:\qtlab_replacement\qsweepy\instrument.py 290 startfreq
D:\qtlab_replacement\qsweepy\instrument.py 290 stopfreq
D:\qtlab_replacement\qsweepy\instrument.py 290 span
D:\qtlab_replacement\qsweepy\instrument.py 290 power
D:\qtlab_replacement\qsweepy\instrument.py 290 zerospan
D:\qtlab_replacement\qsweepy\instrument.py 290 channel_index
D:\qtlab_replacement\qsweepy\instrument.py 290 trigger_source
D:\qtlab_replacement\qsweepy\instrument.py 290 timeout
D:\qtlab_replacement\qsweepy\instrument.py 290 status


In [5]:
switch_sample = {1:1, 5:2, 6:3}

# Прицеливание

In [35]:
sample_name = 'Res MISiS Nb 18.07.18 sample #{}'.format(switch_sample[rf_switch.get_ch2_switch()])
resonator_type='notch_port'
min_freq = 6.0e9
max_freq = 9.0e9
data_root = get_location()

save_path = '{0}/{1}/{2}'.format(data_root[0], data_root[1], sample_name)

In [48]:
pna.set_average(1)
pna.set_averages(1)
pna.set_xlim(min_freq, max_freq)
pna.set_nop(32001)
pna.set_bandwidth(0.5e3)
pna.set_power(0)

True

In [63]:
fig, axes=plt.subplots(2,2, figsize=(8,8))
freqs=pna.get_freqpoints()
S21=pna.get_tracedata()
phase = np.unwrap(S21[1])
delay = np.gradient(phase)/(freqs[1]-freqs[0])
#delay_freqs = (np.asarray(freqs[1:])+np.asarray(freqs[:-1]))/2
delay_freqs=freqs
axes[0,0].plot(freqs,10*np.log10(S21[0]**2))
axes[0,1].plot(freqs, phase)
axes[1,0].plot(np.real(S21[0]*np.exp(1j*S21[1])),np.imag(S21[0]*np.exp(1j*S21[1])))
axes[1,1].plot(delay_freqs, delay)
#axes[1,1].set_yscale('log')

In [49]:
sweep.sweep(pna, ([0], lambda x: 0, ''), filename="S21 overview "+sample_name)

A Jupyter Widget

Started at:  Jul 20 2018 11:51:14
2


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2169: RuntimeWarning: invalid value encountered in remainder
  ddmod = mod(dd + pi, 2*pi) - pi
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2170: RuntimeWarning: invalid value encountered in greater
  _nx.copyto(ddmod, pi, where=(ddmod == -pi) & (dd > 0))
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2172: RuntimeWarning: invalid value encountered in less
  _nx.copyto(ph_correct, 0, where=abs(dd) < discont)



Elapsed time: 0 h 1 m 5.19 s


{'S-parameter': (('Frequency',),
  (array([  6.00000000e+09,   6.00009375e+09,   6.00018750e+09, ...,
            8.99981250e+09,   8.99990625e+09,   9.00000000e+09]),),
  array([ 0.01914358+0.02751474j,  0.01991005+0.02696696j,
          0.02066653+0.02637239j, ...,  0.00779823-0.00029094j,
          0.00778157-0.00049595j,  0.00774907-0.0007163j ]),
  {'log': 20},
  ('Hz',))}

In [64]:
threshold = 5e-7

In [65]:
from scipy.signal import detrend
from scipy.signal import find_peaks_cwt
from scipy.signal import argrelextrema
peaks_expected = 0.1
data = S21
data_points = len(data[0])
data_complex = 10**(data[0]/20)*np.exp(1j*detrend(np.unwrap(data[1])))
data_diff = delay
data_diff[data_diff<threshold] = 0
#data_diff[data_diff>-threshold] = 0
data_diff=np.abs(data_diff)
plt.plot(freqs, data_diff)
#peaks = find_peaks_cwt(data_diff, np.linspace(1, 8, 4), noise_perc=100-(100.*peaks_expected)/data_points)
neighborhood_size = int(np.floor(2e6/(freqs[1]-freqs[0])))
peaks = argrelextrema(data_diff, np.greater, order=neighborhood_size)[0]
print ('found {0} peaks'.format(len(peaks)))
peak_freqs = freqs[peaks]
for freq_id, freq in enumerate(peak_freqs):
    print (freq, ',')

found 23 peaks
6102562500.0 ,
6134531250.0 ,
6306187500.0 ,
6308531250.0 ,
6313875000.0 ,
6334031250.0 ,
6455625000.0 ,
6531562500.0 ,
7052531250.0 ,
7061718750.0 ,
7250343750.0 ,
7344468750.0 ,
7422937500.0 ,
7482750000.0 ,
7543031250.0 ,
7681500000.0 ,
7950656250.0 ,
7953093750.0 ,
8122687500.0 ,
8146968750.0 ,
8316281250.0 ,
8519062500.0 ,
8582906250.0 ,


In [69]:
#rescan peaks individually and fit via circlefit
from resonator_tools import circuit

typical_span=3e6
bw=10e3
nop = typical_span/bw
#pna.set_bandwidth(bw)
pna.set_nop(nop)
pna.set_span(typical_span)
pna.pre_sweep()
high_power_fits = []
for freq_id, freq in enumerate(peak_freqs):
    pna.set_centerfreq(freq)
    measurement = pna.measure()['S-parameter']
    points = pna.get_points()['S-parameter'][0][1]
    delay = np.gradient(measurement)
    fitter = circuit.notch_port(f_data = points, z_data_raw=measurement)
    fitter.autofit()
    high_power_fits.append(fitter.fitresults)
    plt.figure(freq_id)
    fitter.plotall()
pna.post_sweep()
high_power_fits = pd.DataFrame(high_power_fits)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:161: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  warnings.warn(message, mplDeprecation, stacklevel=1)
C:\ProgramData\Anaconda3\lib\site-packages\matplotlib\pyplot.py:528: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the r

In [79]:
high_power_fits = pd.DataFrame(high_power_fits)
resonator_freqs = high_power_fits['fr']
resonator_freq_spans = (high_power_fits['fr']/high_power_fits['Ql']*5).tolist()

In [80]:
resonator_freqs[8] = 7.0525e9
resonator_freq_spans[8] = 5e6
resonator_freq_spans[9] = 5e6
resonator_freq_spans[22] = 8.583e6
resonator_freq_spans[22] = 10e6

# Мерить зонаторы

In [181]:
resonator_freqs = [5418781250.0 ,
5520781250.0 ,
5610312500.0 ,
5706500000.0 ,
5796218750.0 ,
5905437500.0 ,
6003687500.0 ,
6105031250.0 ,
6196625000.0 ,
6302375000.0 ,
6390031250.0 ,
6486125000.0 ,
6572843750.0 ,
6688437500.0 ,
6786593750.0 ,
6879968750.0 ,
6983093750.0 ,
7088375000.0 ,
7172093750.0 ,
7271000000.0 ,
7354718750.0 ,
7471906250.0 ,
7561156250.0 ,
7664375000.0]


In [42]:
resonator_freqs = [ 
6.6532e9
6.8037e9,    
]
#powers = np.linspace(-10, -45, 36) # длинный скан
#powers = np.linspace(-20, -40, 41) # короткий скан

In [89]:
#powers = np.linspace(0, -30, 31) # короткий скан
powers = np.linspace(0, -65, 66) # длинный скан

In [90]:
resonator_ids = [i for i,f in enumerate(resonator_freqs)]
#resonator_ids = np.asarray([22, 23])-1
resonator_ids

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22]

In [91]:
len(resonator_freqs)

23

In [92]:
resonator_freq_span = 1e6
#resonator_freq_spans = None

for resonator_id, resonator_freq in enumerate(resonator_freqs):
    if resonator_id not in resonator_ids:
        continue
    if resonator_freq_spans:
        resonator_freq_span = resonator_freq_spans[resonator_id]
    pna.set_xlim(resonator_freq-resonator_freq_span/2, resonator_freq+resonator_freq_span/2)

    pna.set_nop(101)
    pna.set_average(1)
    pna.set_averages(30)
    
    
    pna.set_bandwidth(1)
    pna._visainstrument.write("SENS:AVER:MODE POIN")

    ref_pow = np.array(	[-80.,	-30.,	20.]	)
    ref_bw = np.array(	[1.,	100000.,	100000.]	)
    def set_power(power):
        bw = 10**np.interp(power, ref_pow, np.log10(ref_bw))
        pna.set_bandwidth(bw)
        pna.set_power(power)
        
    measurement = sweep.sweep_vna(pna, (powers, set_power, 'Power'), filename=sample_name+' res {0}'.format(resonator_id))
    
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type=resonator_type)
    default_measurement_save_path(save_path, mkdir=True)
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    plt.close()
    plt.close()
    #print(fitdata)

A Jupyter Widget

Started at:  Jul 20 2018 12:13:18
2

Elapsed time: 0 h 5 m 48.69 s


A Jupyter Widget

Started at:  Jul 20 2018 12:19:10
2

Elapsed time: 0 h 5 m 48.47 s


A Jupyter Widget

...

A Jupyter Widget

Started at:  Jul 20 2018 14:16:36
2

Elapsed time: 0 h 5 m 48.97 s


C:\ProgramData\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:427: RuntimeWarning: Number of calls to function has reached maxfev = 600.
  warnings.warn(errors[info][0], RuntimeWarning)


A Jupyter Widget

Started at:  Jul 20 2018 14:22:29
2

Elapsed time: 0 h 5 m 49.01 s


# load last measurements and refit

In [17]:
measurement[1]

':'

In [18]:
import glob
measurement_pattern = 'D:\\qtlab\\ReiData\\data\\2018-04-28\\*-CHeGe Vanya-3 res {}\\*.pkl'
for resonator_id in range(3):
    measurements = glob.glob(measurement_pattern.format(resonator_id))
    if len(measurements):
        last_measurement = measurements[-1]
        #print (last_measurement)
        with open(last_measurement, 'rb') as f:
            measurement = pickle.load(f)[1]['S-parameter']
            powers = measurement[1][0]
            fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type='reflection_port')
            fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
            plt.close()
            plt.close()

In [19]:
resonator_freq_span = 1.0e6
resonator_freq_spans = None

for resonator_id, resonator_freq in enumerate(resonator_freqs):
    if resonator_id not in resonator_ids:
        continue
    if resonator_freq_spans:
        resonator_freq_span = resonator_freq_spans[resonator_id]
    pna.set_xlim(resonator_freq-resonator_freq_span/2, resonator_freq+resonator_freq_span/2)

    pna.set_nop(101)
    pna.set_average(1)
    pna.set_averages(10)
    pna.set_bandwidth(1)

    ref_pow = np.array(	[-80.,	-30.,	20.]	)
    ref_bw = np.array(	[1.,	100000.,	100000.]	)
    def set_power(power):
        bw = 10**np.interp(power, ref_pow, np.log10(ref_bw))
        pna.set_bandwidth(bw)
        pna.set_power(power)
        
    measurement = sweep.sweep_vna(pna, (powers, set_power, 'Power'), filename=sample_name+' res {0}'.format(resonator_id))
    
    fitdata = resonator_quality_factor_fit.resonator_quality_factor_fit(measurement, powers, resonator_type=resonator_type)
    fitdata['single_photon_limit'] =  port1.get_single_photon_limit()
    fitdata.to_csv(save_path+'/'+'fit_total {0}.csv'.format(resonator_id))
    plt.close()
    plt.close()
    #print(fitdata)

A Jupyter Widget

Started at:  Jul 19 2018 20:04:21
2


FileNotFoundError: [Errno 2] No such file or directory: 'd:\\data/2018-07-19/20-04-21-Res MISiS Nb 18.07.18 sample #1 res 0/'

# Загрузить фиты и построить графики

In [ ]:
from matplotlib.backends.backend_pdf import PdfPages
fit_csvs = ['{0}/fit_total {1}.csv'.format(save_path, i) for i in range(len(resonator_freqs))]
rows = 4
columns = 1
figsize=(8.27, 11.69)
figs, axes = plt.subplots(rows,columns, figsize=figsize)
pdf = PdfPages('{0}/{1}.pdf'.format(save_path, sample_name))
Q_i_values_lp = []
Q_i_values_hp = []
Q_e_values = []
Q_l_values_lp = []
Q_l_values_hp = []
f_r_values = []
single_photon_limits = []
if resonator_type=='reflection_port':
    Qc_column = 'Qc'
    Qi_column = 'Qi'
    Qi_err_column = 'Qi_err'
    Ql_column = 'Ql'
else:
    Qc_column = 'Qc_dia_corr'
    Qi_column = 'Qi_dia_corr'
    Qi_err_column = 'Qi_dia_corr_err'
    Ql_column = 'Ql'
page_id = 0
for resonator_id, fit_csv in enumerate(fit_csvs):
    try:
        fitdata = pd.read_csv(fit_csv)
        #print (fitdata.shape, powers.shape)
        #if fitdata.shape[0]<len(powers):
        #fitdata['power'] = powers[(len(powers)-fitdata.shape[0]):]
        #fitdata['power'] = powers
        print (fitdata, resonator_id)
        fitdata = fitdata[fitdata['fr']>4e9]
        fitdata = fitdata[fitdata['fr']<12e9]
        fitdata = fitdata[fitdata[Qc_column]>100]
        fitdata = fitdata[fitdata[Qc_column]<1000000]
        #if resonator_id != 11:
        #fitdata = fitdata[fitdata[Qc_column]<800000]
        #else:
        #   fitdata = fitdata[fitdata['Qc_dia_corr']<2000000]
        if resonator_id and not resonator_id%rows*columns: 
            page_id +=1
            plt.tight_layout()
            pdf.savefig(figs)
            figs, axes = plt.subplots(rows, columns, figsize=figsize)
        if rows>1 and columns>1:
            Q_ax = axes[(resonator_id%(rows*columns))%rows, (resonator_id%(rows*columns))//rows]
        else:
            Q_ax = axes[resonator_id%(rows*columns)]
        Q_ax.set_yscale('log')
        Ql = Q_ax.errorbar(fitdata['power'], fitdata['Ql'], label='Q_l')#, yerr=fitdata['Ql_err'])
        Qc = Q_ax.errorbar(fitdata['power'], fitdata[Qc_column], label='Q_c')#, yerr=fitdata['absQc_err'])
        Qi = Q_ax.errorbar(fitdata['power'], np.abs(fitdata[Qi_column]), label='Q_i')#, yerr=fitdata['Qi_dia_corr_err'])
        f_ax = Q_ax.twinx()
        f = f_ax.errorbar(fitdata['power'], fitdata['fr']/1e6, color='orange', marker='.', label='$f$, MHz')#, yerr=fitdata['fr_err'])
        f_ax.set_ylabel('MHz')
        Q_ax.set_ylabel('$Q$')
        if not (fitdata['power'].size == 0):
            Q_i_values_lp.append(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])])
            Q_i_values_hp.append(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])])
            Q_e_values.append(np.median(fitdata[Qc_column]))
            Q_l_values_lp.append(np.abs(fitdata[Ql_column])[np.argmin(fitdata['power'])])
            Q_l_values_hp.append(np.abs(fitdata[Ql_column])[np.argmax(fitdata['power'])])
            f_r_values.append(np.median(fitdata['fr']))
            single_photon_limits.append(fitdata['single_photon_limit'][np.argmax(fitdata['power'])])
            ann = f_ax.annotate('#{7}: $f_r$: {0:2.4} GHz, $Q_i$: {1:2.3g}/{3:2.3g}\n$Q_l$:{5:2.3g}/{6:2.3g}, $Q_c$: {4:2.3g}'.format(np.median(fitdata['fr'])/1e9, \
                            np.round(np.abs(fitdata[Qi_column])[np.argmin(fitdata['power'])]), np.round(fitdata[Qi_err_column][np.argmin(fitdata['power'])]), \
                            np.round(np.abs(fitdata[Qi_column])[np.argmax(fitdata['power'])]), np.median(fitdata[Qc_column]), \
                            np.round(np.abs(fitdata[Ql_column])[np.argmin(fitdata['power'])]), np.round(np.abs(fitdata[Ql_column])[np.argmax(fitdata['power'])]), \
                            resonator_id), \
                          (0.2, 0.1), xycoords='axes fraction', bbox={'alpha':0.5, 'pad':3, 'edgecolor':'black', 'facecolor':'white'})
        Q_ax.set_ylim([1e3, 1e6])
        plt.grid(True)
        if (resonator_id==0):
            leg = plt.legend([f, Qc, Qi, Ql], ['$f$', '$Q_c$', '$Q_i$', '$Q_l$'], fancybox=True, loc=6, framealpha=1)
            leg.get_frame().set_alpha(0.5)
            plt.title(sample_name)
    finally:
        pass
#axes[-1,-1].hist([Q_i_values_hp, Q_i_values_lp, Q_e_values], bins=10, label={'$Q_i$ high power', '$Q_i$ low power', '$Q_e$'})
#axes[-1,-1].set_xlabel('$Q_i$')
#axes[-1,-1].set_ylabel('resonators')
#axes[-1,-1].legend(loc='upper right')
plt.tight_layout()
pdf.savefig(figs)
pdf.close()
summary_table = pd.DataFrame({'fr':f_r_values, 'Ql_lp':Q_l_values_lp, 'Ql_hp':Q_l_values_hp, 'Qe': Q_e_values, 'Qi_lp':Q_i_values_lp, 'Qi_hp':Q_i_values_hp, 'single_photon_limits':single_photon_limits})
summary_table.to_csv('{0}/{1}.csv'.format(save_path, sample_name))

    Unnamed: 0    Qc_dia_corr    Qi_dia_corr  Qi_dia_corr_err     Qi_no_corr  \
0            0  151960.219773   48021.140133      4296.882386   57443.673194   
1            1  148211.065381   50384.082650      5097.341848   60386.119285   
2            2  146556.912393   53114.898218      5747.738941   63966.661523   
3            3  145650.907135   56108.836796      5233.350341   67977.000770   
4            4  143433.951671   59240.602049      5158.351042   72064.604050   
5            5  141889.960998   62667.785601      6397.400419   76668.338930   
6            6  141501.947587   66600.790346      8409.048250   82114.011854   
7            7  140860.827916   70383.923769      6663.518796   87375.587729   
8            8  139576.151324   74626.332378     12352.063546   93281.457548   
9            9  138733.464610   78831.044608      7640.767560   99195.789779   
10          10  138701.271105   83643.984435     13199.111459  106367.232954   
11          11  138856.126612   88907.35

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'argmin' is deprecated. Use 'idxmin' instead. The behavior of 'argmin' will be corrected to return the positional minimum in the future. Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:57: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


 3
    Unnamed: 0   Qc_dia_corr   Qi_dia_corr  Qi_dia_corr_err     Qi_no_corr  \
0            0  84239.591234  30328.662842      2772.396950   34534.714252   
1            1  83239.851721  31800.185362      3336.103309   36297.863212   
2            2  82304.796286  33508.551307      4091.813955   38304.448191   
3            3  81961.957841  35305.691549      2930.159579   40497.788048   
4            4  81198.961600  37221.080909      2814.273250   42787.822911   
5            5  80886.046266  39299.160684      4752.546326   45342.561538   
6            6  80616.226242  41485.629489      3350.865056   48059.761965   
7            7  80550.578429  43821.227712      5415.204866   50962.709326   
8            8  79789.397076  46190.360429      3522.983231   53874.581323   
9            9  79991.435403  48691.750951      6124.403513   57129.707397   
10          10  79899.557050  51263.557176      2927.509006   60444.812198   
11          11  79784.882474  53832.689206      4971.400818  

    Unnamed: 0    Qc_dia_corr   Qi_dia_corr  Qi_dia_corr_err    Qi_no_corr  \
0            0  221175.021312  45371.066023     13263.789351  45435.361449   
1            1  214155.313389  46042.889943     17530.390392  46091.504263   
2            2  209899.988781  47527.259040     20448.664121  47574.334667   
3            3  207837.841826  49651.124920     15818.266215  49706.606614   
4            4  192285.808096  48746.666403     26941.512116  48758.137103   
5            5  148433.523166  41240.258139      2752.648798  41313.702262   
6            6   89054.456667  29012.293675      3218.200360  29207.370736   
7            7   91559.357710  31051.138825      3389.608870  31305.500310   
8            8   93422.708999  33057.914583      3731.640413  33386.095436   
9            9   94102.445364  34665.461725      3702.541897  35065.486684   
10          10   97030.834868  37394.133665      3827.083919  37874.205988   
11          11   99318.271978  40072.699702      3774.327443  40

    Unnamed: 0   Qc_dia_corr    Qi_dia_corr  Qi_dia_corr_err    Qi_no_corr  \
0            0  22542.941871  -25346.770797     4.837562e+04 -11869.335907   
1            1  22788.092077  -25642.906485     5.126742e+04 -11918.364773   
2            2  22910.390706  -25775.294217     5.385622e+04 -11970.833654   
3            3  23410.946587  -26411.495745     5.756620e+04 -12012.980771   
4            4  23656.072721  -26676.752519     6.504755e+04 -12143.925785   
5            5  22760.787663  -25635.263326     4.852992e+04 -11867.014213   
6            6  22226.859439  -24969.548172     4.404487e+04 -11768.762442   
7            7  22660.822134  -25496.614195     4.847091e+04 -11905.824905   
8            8  24039.372032  -27203.711778     6.343846e+04 -12116.505738   
9            9  22755.280528  -25592.487085     5.065738e+04 -11967.219976   
10          10  22391.671399  -25205.036747     4.282551e+04 -11816.788258   
11          11  21209.250324  -23770.530915     3.487785e+04 -11

In [86]:
import math
summary_table_new = pd.DataFrame(summary_table)
summary_table_new['k'] = summary_table_new['fr']/summary_table_new['Qe']
(summary_table_new['Ql_lp']).round(decimals=0).astype(int)

0      69030
1      97932
2     105153
3      52813
4      40684
5      43634
6      80742
7     109205
8      47150
9        461
10     12764
11      7131
12     78771
13     14304
14     35118
15     35378
16     54722
17     46054
18     65986
19     32981
20    129688
21      7752
22      1791
Name: Ql_lp, dtype: int32

In [87]:
import math
summary_table_new = pd.DataFrame(summary_table)
summary_table_new['k'] = summary_table_new['fr']/summary_table_new['Qe']
summary_table_new

,Qe,Qi_hp,Qi_lp,Ql_hp,Ql_lp,fr,single_photon_limits,k
0,136968.395817,47885.194524,149190.350547,36359.056129,69030.123110,6.102553e+09,-143.537110,4.455446e+04
1,120806.695943,478902.796534,530093.482244,96471.150203,97932.224013,6.134485e+09,-152.937732,5.077935e+04
2,212392.398790,51457.783105,208191.842052,40298.376309,105153.497037,6.306313e+09,-143.245689,2.969180e+04
3,128894.794456,30264.968366,95150.385953,25060.249922,52812.623685,6.308644e+09,-140.172183,4.894413e+04
4,78904.602137,30294.599326,89057.121090,22277.551261,40684.260833,6.313999e+09,-141.525529,8.002066e+04
5,112371.197938,37170.580286,71472.881872,30789.143838,43633.992885,6.334203e+09,-141.023846,5.636857e+04
6,162066.901064,48621.394226,158516.477657,35497.434207,80742.183139,6.455800e+09,-143.441772,3.983417e+04
7,364351.572353,69349.766754,155576.270497,60007.337132,109205.142704,6.531713e+09,-142.602001,1.792695e+04
8,22569.435506,25630.192123,4982.949971,205649.836965,47149.868179,7.054128e+09,-165.542521,3.125522e+05
9,1048.024993,17120.972213,15538.340818,985.561825,460.933400,7.062027e+09,-132.527008,6.738414e+06


In [88]:
figs, ax = plt.subplots(1,4, figsize=(16, 5))
c = 'fr'
ax[0].plot(summary_table_new[c], marker='o')
ax[0].set_xlabel('resonator id\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[0].set_ylabel('$f_r$')
ax[0].grid(True, which='both')
c = 'Qe'
ax[1].boxplot(summary_table_new['Qe'])
ax[1].set_yscale('log')
ax[1].grid(True, which='both')
ax[1].set_ylabel('$Q_e$')
ax[1].set_xlabel('\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[1].set_ylim(1e3, 1e6)
c = 'Qi_hp'
ax[2].boxplot(summary_table_new['Qi_hp'])
ax[2].set_ylabel('$Q_i$ (hp)')
ax[2].set_yscale('log')
ax[2].grid(True, which='both')
ax[2].set_xlabel('\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[2].set_ylim(1e3, 1e6)
c = 'Qi_lp'
ax[3].boxplot(summary_table_new['Qi_lp'])
ax[3].set_ylabel('$Q_i$ (lp)')
ax[3].set_yscale('log')
ax[3].grid(True, which='both')
ax[3].set_xlabel('\nmin: {0:6.4g}\n25%: {1:6.4g}\n50%: {2:6.4g}\n75%: {3:6.4g}\nmax: {4:6.4g}'.format(
    summary_table_new[c].min(),
    summary_table_new[c].quantile(0.25),
    summary_table_new[c].quantile(0.5),
    summary_table_new[c].quantile(0.75),
    summary_table_new[c].max(),
                ))
ax[3].set_ylim(1e3, 1e6)
plt.tight_layout()
plt.suptitle(sample_name)
plt.savefig('{0}/{1}-boxplot.pdf'.format(save_path, sample_name))

In [143]:
print(summary_table_new['fr'].describe())

count    2.300000e+01
mean     6.471979e+09
std      6.745989e+08
min      5.389021e+09
25%      5.922199e+09
50%      6.468419e+09
75%      7.017835e+09
max      7.638156e+09
Name: fr, dtype: float64


In [229]:
rf_switch.get_ch1_switch()

0

# bullshit

In [23]:
freqs=measurement.get_data()[1]
powers=measurement.get_data()[0]
max_power_S21 = 10**(measurement.get_data()[2][np.argmax(powers), :]/20)*np.exp(1j*measurement.get_data()[3][np.argmax(powers), :])

fitter = circuit.reflection_port(f_data=freqs, z_data_raw=max_power_S21)
print(fitter.do_calibration(freqs, max_power_S21))
fitter.autofit()
fitter.plotall()
print(fitter.fitresults)
print(fitter.get_single_photon_limit())

(8.7617753961513634e-08, 0.00021846153804807672, 0.78230385058357799, 9473514161.3102646, 987237.85108576983, 0.0, 9480474204.3878784)
{'theta0': -0.87036269078204709, 'Qi_err': 4279766.7041571699, 'Ql_err': 109.19720855604572, 'Qc_err': 91.415426058562019, 'fr': 9482648929.5436897, 'Qc': 763.26702083792111, 'Qi': 198307.89384138637, 'fr_err': 516643.97835309943, 'Ql': 760.34054699522324, 'chi_square': 1.0765021049802501}
-129.081045603


In [6]:
measurement = dm.load_measurement('C:/qtlab/ReiData/data/Feb 27 2017/16-17-54/Res Nb CEXP 17.01.2016 switch 4-2 SH A anti-korenkovian warm')